In [71]:
# 导入依赖库
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored 
from openai import OpenAI
from crossref.restful import Works
GPT_MODEL = "llama3.1:8b"
client = OpenAI( api_key="ollama", 
    base_url="http://127.0.0.1:11434/v1",)  


In [72]:

@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [73]:
tools=[{
		"type": "function", 
		"function": { 
			"name": "get_paper_crossref_api", 
			"description": """ 
				Search for articles, call this tool when a user requests you to search for articles.
			""",
			"parameters": { # 使用 parameters 字段来定义函数接收的参数
				"type": "object",
				"required": ["search_name","author_name","start_year","end_year"], # 使用 required 字段告诉 Kimi 大模型哪
				"properties": {
                    "search_name": {
                        "type": "string",
                        "description": "结合上下文，用户要查找的论文的主题",
                    },
                    "author_name": {
                        "type": "string",
                        "description": "结合上下文，用户要查找的论文的作者,如果没有，返回'None'",
                    },
                    "start_year": {
                        "type": "string",
                        "description": "结合上下文，用户要查找的论文起始年份，如果没有，返回'None'",
                    },
                    "end_year": {
                        "type": "string",
                        "description": "结合上下文，用户要查找的论文的截止年份,如果没有，返回'None'",
                    }, 
                     "num":{
                        "type": "int",
                        "description": "结合上下文，用户要查找的论文数量,如果没有，返回int 3",
                    }
                },
			}
		}
	},]

In [74]:
works = Works()
def get_paper_crossref_api(search_name,author_name,start_year,end_year,num):
    print(search_name, author_name, start_year, end_year)
    json_file=[]
    count=0
    num=int(num)
    for item in works.query(search_name).sort('relevance').order('desc').filter(has_license='true').select('title', 'DOI',"author","abstract"):
        count += 1
        print(item)
        json_file.append(item)
        if count > num:
            break
    if author_name!="None":
        json_file=[]
        for item in works.query(search_name,author=author_name).sort('relevance').order('desc').filter(has_license='true').select('title', 'DOI',"author","abstract"):
            count += 1
            print(item)
            json_file.append(item)
            if count > num:
                break
        if start_year!="None" and end_year!="None":
            json_file=[]    
            for item in works.query(search_name,author=author_name).sort('relevance').order('desc').filter(from_pub_date=start_year, until_pub_date=end_year,has_license='true').select('title', 'DOI',"author","abstract"):
                count += 1
                print(item)
                json_file.append(item)
                if count > num:
                    break  

    
    print(json_file)
    return json_file

In [75]:
messages = []
# messages.append({"role": "system", "content": "不要假设应该把什么值代入函数中。如果用户的请求模棱两可，要求对方澄清。"})
messages.append({"role": "user", "content": "查找6篇有关carbon materials的文章  "})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_paper_crossref_api"}}
)
response_message=chat_response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    print("use tools")
    # 步骤3：调用函数
    # 注意：JSON响应可能并不总是有效；一定要处理错误
    available_functions = {
        "get_paper_crossref_api": get_paper_crossref_api,
    }  # 这个示例中只有一个函数，但你可以有多个函数
    messages.append(response_message)  # 用助手的回复扩展对话
    # 步骤4：发送每个函数调用的信息和函数响应给模型
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            search_name=function_args.get("search_name"),
            author_name=function_args.get("author_name"),
            start_year=function_args.get("start_year"),
            end_year=function_args.get("end_year"),
            num=function_args.get("num")
        )
        print("function_response:",function_response)
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content":str(function_response)+ "Organize all results seperately,only back title,doi,summary"
            }
        )  # 用函数响应扩展对话
    second_response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=messages,
    )  # 获取模型的新响应，其中可以看到函数响应
print(second_response.choices[0].message.content)


use tools
carbon materials None None None
{'DOI': '10.1016/b978-0-8155-1324-7.50011-1', 'author': [{'given': 'John J.', 'family': 'Kibler', 'sequence': 'first', 'affiliation': []}], 'title': ['Mechanics of Multidirectional Carbon-Carbon Composite Materials']}
{'DOI': '10.1016/b978-0-8155-1324-7.50010-x', 'author': [{'given': 'N.', 'family': 'Murdie', 'sequence': 'first', 'affiliation': []}, {'given': 'C.P.', 'family': 'Ju', 'sequence': 'additional', 'affiliation': []}, {'given': 'J', 'family': 'Don', 'sequence': 'additional', 'affiliation': []}, {'given': 'M.A.', 'family': 'Wright', 'sequence': 'additional', 'affiliation': []}], 'title': ['Carbon-Carbon Matrix Materials']}
{'DOI': '10.1016/s1872-5805(08)60048-7', 'author': [{'given': 'Michio', 'family': 'Inagaki', 'sequence': 'first', 'affiliation': []}], 'title': ['Pores in carbon materials-importance of their control']}
{'DOI': '10.1016/s1872-5805(23)60736-x', 'author': [{'given': 'Michio', 'family': 'Inagaki', 'sequence': 'first', '

In [76]:
print(second_response.choices[0].message.content)

Here are the 6 articles related to carbon materials, organized separately with only their titles, DOIs, and summaries:

**Article 1:**
* Title: "Mechanics of Multidirectional Carbon-Carbon Composite Materials"
* DOI: 10.1016/b978-0-8155-1324-7.50011-1
* Summary: This article discusses the mechanics of multidirectional carbon-carbon composite materials.

**Article 2:**
* Title: "Carbon-Carbon Matrix Materials"
* DOI: 10.1016/b978-0-8155-1324-7.50010-x
* Summary: This article provides an overview of carbon-carbon matrix materials.

**Article 3:**
* Title: "Pores in carbon materials-importance of their control"
* DOI: 10.1016/s1872-5805(08)60048-7
* Summary: This article highlights the importance of controlling pores in carbon materials.

**Article 4:**
* Title: "Carbon materials for water desalination by capacitive deionization"
* DOI: 10.1016/s1872-5805(23)60736-x
* Summary: This article explores the use of carbon materials for water desalination through capacitive deionization.

**Arti